In [1]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("test").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/11 22:54:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/12/11 22:54:19 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [4]:
schema = StructType([
    StructField("stock_name", StringType(), True),
    StructField("operation", StringType(), True),
    StructField("operation_day", IntegerType(), True),
    StructField("price", IntegerType(), True)
])

# Data to be loaded into the DataFrame
data = [
    ("Leetcode", "Buy", 1, 1000),
    ("Corona Masks", "Buy", 2, 10),
    ("Leetcode", "Sell", 5, 9000),
    ("Handbags", "Buy", 17, 30000),
    ("Corona Masks", "Sell", 3, 1010),
    ("Corona Masks", "Buy", 4, 1000),
    ("Corona Masks", "Sell", 5, 500),
    ("Corona Masks", "Buy", 6, 1000),
    ("Handbags", "Sell", 29, 7000),
    ("Corona Masks", "Sell", 10, 10000)
]

# Create DataFrame
df = spark.createDataFrame(data, schema)

# Show the DataFrame
df.show()

print(df.dtypes)

+------------+---------+-------------+-----+
|  stock_name|operation|operation_day|price|
+------------+---------+-------------+-----+
|    Leetcode|      Buy|            1| 1000|
|Corona Masks|      Buy|            2|   10|
|    Leetcode|     Sell|            5| 9000|
|    Handbags|      Buy|           17|30000|
|Corona Masks|     Sell|            3| 1010|
|Corona Masks|      Buy|            4| 1000|
|Corona Masks|     Sell|            5|  500|
|Corona Masks|      Buy|            6| 1000|
|    Handbags|     Sell|           29| 7000|
|Corona Masks|     Sell|           10|10000|
+------------+---------+-------------+-----+

[('stock_name', 'string'), ('operation', 'string'), ('operation_day', 'int'), ('price', 'int')]


In [5]:
df.createOrReplaceTempView("Stocks")

In [9]:
result = spark.sql(
    """          
    select *, lead(price) over(partition by stock_name order by operation_day) - price as profit
    from Stocks

    """
)
result.show()

+------------+---------+-------------+-----+------+
|  stock_name|operation|operation_day|price|profit|
+------------+---------+-------------+-----+------+
|Corona Masks|      Buy|            2|   10|  1000|
|Corona Masks|     Sell|            3| 1010|   -10|
|Corona Masks|      Buy|            4| 1000|  -500|
|Corona Masks|     Sell|            5|  500|   500|
|Corona Masks|      Buy|            6| 1000|  9000|
|Corona Masks|     Sell|           10|10000|  NULL|
|    Handbags|      Buy|           17|30000|-23000|
|    Handbags|     Sell|           29| 7000|  NULL|
|    Leetcode|      Buy|            1| 1000|  8000|
|    Leetcode|     Sell|            5| 9000|  NULL|
+------------+---------+-------------+-----+------+



In [11]:
result = spark.sql(
    """          
    select stock_name, sum(profit) as capital_gain_loss
    from (select *, lead(price) over(partition by stock_name order by operation_day) - price as profit
        from Stocks) p 
    where operation = "Buy"
    group by 1
    """
)
result.show()

+------------+-----------------+
|  stock_name|capital_gain_loss|
+------------+-----------------+
|Corona Masks|             9500|
|    Handbags|           -23000|
|    Leetcode|             8000|
+------------+-----------------+



In [13]:
result = spark.sql(
    """          
    select stock_name, sum(case when operation = "Buy" then -price else price end) as capital_gain_loss
    from Stocks
    group by 1
    """
)
result.show()

+------------+-----------------+
|  stock_name|capital_gain_loss|
+------------+-----------------+
|    Leetcode|             8000|
|Corona Masks|             9500|
|    Handbags|           -23000|
+------------+-----------------+

